In [1]:
import time
import random
from datetime import datetime

import selenium
import webdriver_manager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

from crawler import get_url

In [ ]:
wd.quit()

In [2]:
variables = {}

In [3]:
url = "https://v2.quantus.kr/backtest/universe"
# url = "v2.quantus.kr/backtest/universe"

wd = get_url(url, window=True, image=True, logging=True)

In [4]:
wd.get_log(log_type="browser")

[]

In [5]:
for req in wd.requests:
    res = req.response
    print(res.status_code)
    print(res.headers)
    # print(res.body)
    

200
Server: nginx/1.18.0 (Ubuntu)
Date: Tue, 03 Jan 2023 09:52:33 GMT
Content-Type: text/html; charset=utf-8
Transfer-Encoding: chunked
Connection: keep-alive
Access-Control-Allow-Origin: *
Access-Control-Allow-Methods: GET
Content-Encoding: gzip


200
content-type: application/json; charset=utf-8
access-control-allow-origin: https://www.google.com
access-control-allow-credentials: true
x-content-type-options: nosniff
cache-control: no-cache, no-store, max-age=0, must-revalidate
pragma: no-cache
expires: Mon, 01 Jan 1990 00:00:00 GMT
date: Tue, 03 Jan 2023 09:52:33 GMT
strict-transport-security: max-age=31536000; includeSubDomains
content-security-policy: script-src 'report-sample' 'nonce-M9wgExUsNuMJvXFa5rA9bA' 'unsafe-inline';object-src 'none';base-uri 'self';report-uri /_/IdentityListAccountsHttp/cspreport;worker-src 'self'
content-security-policy: script-src 'unsafe-inline' 'self' https://apis.google.com https://ssl.gstatic.com https://www.google.com https://www.gstatic.com https:/

In [6]:
# input data: level (i)
# i = 0: 초급
# i = 1: 중급
# i = 2: 고급

d = {
    0: "초급",
    1: "중급",
    2: "고급",
}
keys = list(d.keys())
i = random.randint(min(keys), max(keys))
# variables["level"] = d[i]  

# select level
levels = wd.find_elements(By.CLASS_NAME, "css-2ho4nc")
levels[i].click()

# apply
wd.find_element(By.CLASS_NAME, "yes").click()
wd.find_element(By.CLASS_NAME, "css-1lcj2w").click()

In [7]:
# click login button
login_xpath = "/html/body/div[1]/div/div[1]/div/div[1]/div/div/div/p"
wd.find_element(By.XPATH, login_xpath).click()

# input data: login_method (i)
# i = 1: google
# i = 2: kakao
d = {
    0: "구글",
    1: "카카오",
}
keys = list(d.keys())
i = random.randint(min(keys), max(keys))
login = d[i]
variables["login"] = login

# google_login_xpath = "/html/body/div[1]/div/div[2]/div/div/div/a[1]"
# kakao_login_xpath = "/html/body/div[1]/div/div[2]/div/div/div/a[2]"
# login_method_xpath = f"/html/body/div[1]/div/div[2]/div/div/div/a[{i}]"

wd.find_element(By.ID, login).click()

In [8]:
variables

{'login': '구글'}

In [9]:
# Enter the login info
email = 'wjddustjsla@naver.com'
pw = 'jys1013011!'

elm_email = wd.find_element(By.ID, "input-loginKey")
elm_pw = wd.find_element(By.ID, "input-password")

# input login info
elm_email.send_keys(email)
elm_pw.send_keys(pw)

# click login button
# wd.find_element(By.CLASS_NAME, "btn_g highlight").click()
wd.find_element(By.XPATH, "/html/body/div/div/div/main/article/div/div/form/div[4]/button[1]").click()

In [10]:
# input data: level (i)
# i = 0: 초급
# i = 1: 중급
# i = 2: 고급

d = {
    0: "초급",
    1: "중급",
    2: "고급",
}
keys = list(d.keys())
i = random.randint(min(keys), max(keys))

## for test: i = 2 (고급)
i = 2
variables["level"] = d[i]

# select level
levels = wd.find_elements(By.CLASS_NAME, "css-2ho4nc")
levels[i].click()

# apply
wd.find_element(By.CLASS_NAME, "yes").click()
wd.find_element(By.CLASS_NAME, "css-1lcj2w").click()

In [11]:
# strategy name
now = datetime.now().strftime("%y%m%d_%H_%M_%S")
strategy_name = f"test_{now}"
print(f"Strategy name: {strategy_name}")

# Enter the strategy name
strategy_name_xpath = "/html/body/div[1]/div/div[2]/div[1]/div/div[1]/input"
elm_strategy_name = wd.find_element(By.XPATH, strategy_name_xpath)
elm_strategy_name.send_keys(strategy_name)
variables["strategy_name"] = strategy_name

Strategy name: test_230103_18_52_57


In [12]:
# 유티버스 선택

"""

0: 한국
1: 미국
2: KOSPI
3: KOSDAQ
"""
d = {
    0: "한국",
    1: "미국",
    2: "KOSPI",
    3: "KOSDAQ",
}
keys = list(d.keys())
i = random.randint(min(keys), max(keys))
variables["universe"] = d[i]
print(f"Universe: {d[i]}")


wd.find_element(By.CLASS_NAME, "css-cy3vpx").click()
universes = wd.find_elements(By.CLASS_NAME, "css-zf30n6")

universe = universes[i]
universe.click()

Universe: 미국


In [13]:
variables

{'login': '구글',
 'level': '고급',
 'strategy_name': 'test_230103_18_52_57',
 'universe': '미국'}

In [14]:
soup = BeautifulSoup(wd.page_source, "lxml")
filter_sector_class = "css-192m3r6"

filters = soup.find("div", "css-i2b1or").find_all("div", filter_sector_class)
len_filters = len(filters)
sectors = soup.find("div", "css-16ww4h3").find_all("div", filter_sector_class)

# 기본 필터 & 제외할 섹터
elm_filter_sector = wd.find_elements(By.CLASS_NAME, filter_sector_class)
# # filter elements
# elm_filters = elm_filter_sector[0:len_filters]
# # sector elements
# elm_sectors = elm_filter_sector[len_filters:]

# check filter
i = 0
variables["removed_filter"] = []
for filter in filters:
    c = round(random.random(), 0)
    if c == 1:
        elm_filter_sector[i].click()
        filter = filter.text
        variables["removed_filter"].append(filter)
    i += 1
    
time.sleep(3)

# check sector 
variables["removed_sector"] = []
for sector in sectors:
    c = round(random.random(), 0)
    if c == 1:
        elm_filter_sector[i].click()
        sector = sector.text
        print(sector)
        variables["removed_sector"].append(sector)
    i += 1

건강관리
화장품,의류,완구
보험
비철,목재 등
건설,건축관련
에너지
기계
철강
반도체
증권
디스플레이
미디어,교육
은행
소프트웨어


In [15]:
variables

{'login': '구글',
 'level': '고급',
 'strategy_name': 'test_230103_18_52_57',
 'universe': '미국',
 'removed_filter': ['지주사 제외',
  '관리종목 제외',
  '적자기업 제외',
  '적자기업 제외 (년간)',
  'PTP 기업 제외'],
 'removed_sector': ['건강관리',
  '화장품,의류,완구',
  '보험',
  '비철,목재 등',
  '건설,건축관련',
  '에너지',
  '기계',
  '철강',
  '반도체',
  '증권',
  '디스플레이',
  '미디어,교육',
  '은행',
  '소프트웨어']}

In [16]:
# click '다음' button
wd.find_element(By.CLASS_NAME, "css-etpc0o").click()

In [17]:
# factor 선택

soup = BeautifulSoup(wd.page_source, "lxml")

factor_area_class = "css-kwx3if"
factor_kind_class = "css-12hmsi2"
factor_elm_class = "css-192m3r6"

factor_area = soup.find_all("div", factor_area_class)
elm_factors = wd.find_elements(By.CLASS_NAME, factor_elm_class)

i = 0
for fs in factor_area:
    factor_kind = fs.find("div", factor_kind_class).text.strip()
    variables[factor_kind] = []
    fs_nms = fs.find_all("div", factor_elm_class)
    for fs_nm in fs_nms:
        c = round(random.random(), 0)
        if c == 1:
            fs_nm = fs_nm.text.strip().replace("?", "")
            variables[factor_kind].append(fs_nm)
            elm_factors[i].click()
            time.sleep(1)
        i += 1

127.0.0.1:49574: Traceback (most recent call last):
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packages/seleniumwire/thirdparty/mitmproxy/net/tcp.py", line 115, in read
    data = self.o.read(rlen)
           ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packages/seleniumwire/thirdparty/mitmproxy/server/server.py", line 113, in handle
    root_layer()
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packages/seleniumwire/thirdparty/mitmproxy/server/modes/http_proxy.py", line 9, in __call__
    layer()
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packag

In [18]:
variables

{'login': '구글',
 'level': '고급',
 'strategy_name': 'test_230103_18_52_57',
 'universe': '미국',
 'removed_filter': ['지주사 제외',
  '관리종목 제외',
  '적자기업 제외',
  '적자기업 제외 (년간)',
  'PTP 기업 제외'],
 'removed_sector': ['건강관리',
  '화장품,의류,완구',
  '보험',
  '비철,목재 등',
  '건설,건축관련',
  '에너지',
  '기계',
  '철강',
  '반도체',
  '증권',
  '디스플레이',
  '미디어,교육',
  '은행',
  '소프트웨어'],
 '가치 팩터 (Price 관련)': ['PBR', 'PGPR', 'PEG', 'PAR', '배당수익률', '주주수익률'],
 '가치 팩터 (EV 관련)': ['EV/Net',
  'EV/EBITDA',
  'EV/EBIT',
  'EV/R&D',
  'EV/CF',
  'EV/AC'],
 '퀄리티 팩터': ['ROA',
  'GP/E',
  'GP/A',
  'GPM',
  'NPM',
  'R&D / 매출액',
  'R&D / 매출총이익',
  'AC/E',
  '변동성 (52주)',
  '변동성 (60일)',
  '영업이익 / 차입금',
  '차입금비율'],
 '가격 팩터': ['3개월 모멘텀',
  '6개월 모멘텀',
  '베타',
  '베타 (60일)',
  '절대값 베타',
  '절대값 베타 (60일)',
  'Turnover Rate'],
 '성장성 팩터': ['매출총이익성장률 (QoQ)',
  '매출액성장률 (YoY)',
  '자본성장률 (YoY)',
  'Earning Acceleration (YoY)',
  'Earning Acceleration (QoQ)']}

In [19]:
# click '다음' button
wd.find_element(By.CLASS_NAME, "css-etpc0o").click()

In [20]:
# consts.py

init_amounts = 2000 # 만원 단위
fees = 1 # %(퍼센트)
stock_counts = 10
stop_loss = 26 # %(퍼센트)
profit = 26 # %(퍼센트)

rebalancing_period, rebalancing_method, rebalancing_strategy = None, None, None

backtest_conditions_input = {
    0: ["초기 투자 금액", init_amounts],
    1: ["거래 수수료", fees],
    # 2: ["리밸런싱 기간", rebalancing_period],
    # 3: ["비중 조절 방법", rebalancing_method],
    2: ["종목 수", stock_counts],
    # 5: ["리밸런싱 전략", rebalancing_strategy],
    3: ["손절기준", stop_loss],
    4: ["익절기준", profit],
}

In [21]:
all_elms_class = "css-cy3vpx" # input box & select box

''' input box
0. 초기 투자 금액 *
1. 거래 수수료 *
2. 종목수 * (20으로 입력되어 있음)
3. 손절기준 (입력안할 시 비활성화)
4. 익절기준 (입력안할 시 비활성화)
'''
input_class = "css-1tmhs6j"

''' select box
5. 리밸런싱 기간 *
6. 비중 조절 방법 * ('동일 비중'으로 지정되어 있음)
7. 리밸런싱 전략 * ('분할 매수/매도'로 지정되어 있음)
'''
select_class = "css-cy3vpx"


In [22]:
all_elms = wd.find_elements(By.CLASS_NAME, all_elms_class)
input_elms = wd.find_elements(By.CLASS_NAME, input_class)
select_elms = wd.find_elements(By.CLASS_NAME, select_class)

soup = BeautifulSoup(wd.page_source, "lxml")
all_boxes = soup.find_all("div", all_elms_class)

i = 0
for elm in input_elms:
    cond, val = backtest_conditions_input[i]
    variables[cond] = val
    elm.clear()
    time.sleep(1.5)
    elm.send_keys(val)
    if i == 1:
        # 리밸런싱 기간 선택
        wd.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[3]/div/div[6]/div[1]/div[2]/input").click()

        d = {
            1: "월별",
            2: "분기별",
            3: "반기별",
            4: "매년",
        }

        keys = list(d.keys())
        i = random.randint(min(keys), max(keys))
        variables["rebalancing_period"] = d[i]

        rebalancing_period_xpath = f"/html/body/div[1]/div/div[2]/div[3]/div/div[6]/div[3]/div[{i}]/div/input"

        wd.find_element(By.XPATH, rebalancing_period_xpath).click()
    
    i += 1

In [ ]:
# # 리밸런싱 기간 선택
# wd.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[3]/div/div[6]/div[1]/div[2]/input").click()

# d = {
#     1: "월별",
#     2: "분기별",
#     3: "반기별",
#     4: "매년",
# }

# keys = list(d.keys())
# i = random.randint(min(keys), max(keys))
# variables["rebalancing_period"] = d[i]

# rebalancing_period_xpath = f"/html/body/div[1]/div/div[2]/div[3]/div/div[6]/div[3]/div[{i}]/div/input"

# wd.find_element(By.XPATH, rebalancing_period_xpath).click()

In [23]:
# save strategy
save_button_class = "css-1o923v9"
wd.find_element(By.CLASS_NAME, save_button_class).click()

In [25]:
# backtest
backtest_button_class = "css-etpc0o"
wd.find_element(By.CLASS_NAME, backtest_button_class).click()

In [24]:
from selenium.common.exceptions import NoSuchElementException

alert_class = "css-1x7ny0l"
alert_check_class = "css-1lcj2w"

try:
    soup = BeautifulSoup(wd.page_source, "lxml")
    if soup.find("div", alert_class):
        alert_msg = soup.find("div", alert_class).text
    wd.find_element(By.CLASS_NAME, alert_check_class).click()
except NoSuchElementException:
    print("BackTest!!")

In [26]:
wd.get_log(log_type="browser")

[{'level': 'SEVERE',
  'message': "https://accounts.kakao.com/login/?continue=https%3A%2F%2Fkauth.kakao.com%2Foauth%2Fauthorize%3Fresponse_type%3Dcode%26redirect_uri%3Dhttps%253A%252F%252Fv2.quantus.kr%252Foauth%252Fcallback%252Fkakao%26through_account%3Dtrue%26client_id%3D9744c3a57c0acc1623f70b5df2d2eb8f - Access to fetch at 'https://logins.daum.net/accounts/endpoint/favicon.ico' from origin 'https://accounts.kakao.com' has been blocked by CORS policy: No 'Access-Control-Allow-Origin' header is present on the requested resource. If an opaque response serves your needs, set the request's mode to 'no-cors' to fetch the resource with CORS disabled.",
  'source': 'javascript',
  'timestamp': 1672739570563},
 {'level': 'SEVERE',
  'message': 'https://logins.daum.net/accounts/endpoint/favicon.ico - Failed to load resource: net::ERR_FAILED',
  'source': 'network',
  'timestamp': 1672739570563},
 {'level': 'WARNING',
  'message': 'https://accounts.kakao.com/login/?continue=https%3A%2F%2Fkauth

In [30]:
for req in wd.requests:
    res = req.response
    if res is not None:
        if res.status_code == 404:
            print("404 Error !!")
            break
    # print(res.body)
    
# if wd.requests[-1].response.status_code == "404":
#     print(1)

127.0.0.1:50146: Traceback (most recent call last):
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packages/seleniumwire/thirdparty/mitmproxy/net/tcp.py", line 115, in read
    data = self.o.read(rlen)
           ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packages/seleniumwire/thirdparty/mitmproxy/server/server.py", line 113, in handle
    root_layer()
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packages/seleniumwire/thirdparty/mitmproxy/server/modes/http_proxy.py", line 9, in __call__
    layer()
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packag

404 Error !!


127.0.0.1:50152: Traceback (most recent call last):
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packages/seleniumwire/thirdparty/mitmproxy/net/tcp.py", line 115, in read
    data = self.o.read(rlen)
           ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packages/seleniumwire/thirdparty/mitmproxy/server/server.py", line 113, in handle
    root_layer()
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packages/seleniumwire/thirdparty/mitmproxy/server/modes/http_proxy.py", line 9, in __call__
    layer()
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packag

In [31]:
wd.requests[-1].response.status_code

200

127.0.0.1:50873: Traceback (most recent call last):
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packages/seleniumwire/thirdparty/mitmproxy/net/tcp.py", line 115, in read
    data = self.o.read(rlen)
           ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packages/seleniumwire/thirdparty/mitmproxy/server/server.py", line 113, in handle
    root_layer()
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packages/seleniumwire/thirdparty/mitmproxy/server/modes/http_proxy.py", line 9, in __call__
    layer()
  File "/Users/yeonseojung/Projects/quantus_v2_qa/venv/lib/python3.11/site-packag

---
#### Test: logging

In [ ]:
import os
import re
import sys
import time
import json
import socket
import requests
# import pickle
import numpy as np
import pandas as pd

# Scrapping
from bs4 import BeautifulSoup
from seleniumwire import webdriver
from user_agent import generate_user_agent
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.alert import Alert
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

def get_url_log(url, window=False, image=False, limit=1):
    ''' Set up webdriver, useragent & Get url '''
    
    wd = None
    socket.setdefaulttimeout(30)
    # url parsing 시도횟수
    attempts = 0 
    # limit회 이상 parsing 실패시 pass
    while attempts < limit:
        # try:  
        attempts += 1
        # user agent
        options = Options() 
        userAgent = generate_user_agent(os=('mac', 'linux'), navigator='chrome', device_type='desktop')
        options.add_argument('window-size=1920x1080')
        options.add_argument("--disable-gpu")
        options.add_argument('--disable-extensions')
        options.add_argument(f'user-agent={userAgent}')
        options.add_argument("--start-fullscreen")
        
        if not window:
            options.add_argument('headless')
        if not image:
            options.add_argument('--blink-settings=imagesEnabled=false')

        dc = DesiredCapabilities.CHROME
        #log 종류는 OFF, SEVERE, WARNING, INFO, DEBUG, ALL 존재
        dc['goog:loggingPrefs'] = {'browser': 'ALL'}
        # web driver 
        wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options, desired_capabilities=dc)
        wd.get(url)
        wd.implicitly_wait(5)
        break

        # # 예외처리
        # except Exception as e:
        #     print(f'\n\nError: {str(e)}\n\n')
            
        #     # time.sleep(60)
        #     try:
        #         wd.quit()
        #     except:
        #         pass
        #     wd = None
    return wd

In [ ]:
wd = get_url_log(url, True, True)

In [ ]:
wd.get_log(log_type="browser")

In [ ]:
_requests = wd.requests

In [ ]:
i = 0
for r in _requests:
    res =  _requests[i].response
    if res is None:
        pass
    elif res.status_code == 404:
        break
    i += 1

In [ ]:
wd.quit()

In [ ]:
# factor 선택

soup = BeautifulSoup(wd.page_source, "lxml")

factor_area_class = "css-kwx3if"
factor_kind_class = "css-12hmsi2"
factor_elm_class = "css-1z2md3"
available_factor_class = "css-192m3r6"

factor_area = soup.find_all("div", factor_area_class)
elm_factors = wd.find_elements(By.CLASS_NAME, factor_elm_class)

i = 0
for fs in factor_area:
    factor_kind = fs.find("div", factor_kind_class).text.strip()
    variables[factor_kind] = []
    fs_nms = fs.find_all("div", factor_elm_class)
    for fs_nm in fs_nms:
        _fs_nm = fs_nm.text.strip().replace("?", "")
        if fs_nm.find("div", available_factor_class):
            variables[factor_kind].append(_fs_nm)
            # elm_factors[i].click()
            # time.sleep(1)
        else:
            variables[factor_kind].append([_fs_nm, "unavailable"])
        i += 1

In [ ]:
variables

In [ ]:
wd = get_url("file:///Users/yeonseojung/Downloads/terms.html")
soup = BeautifulSoup(wd.page_source, "lxml")
soup

In [ ]:
strongs = soup.find_all("strong")

errors = []
terms = {}
for strong in strongs:
    strong = strong.text.strip()
    
    if " = " in strong:
        _strong = strong.split(" = ")
        if len(_strong) == 2:
            terms[_strong[0]] = _strong[1]
        else:
            errors.append(strong)
        
        
    elif ". " in strong:
        _strong = strong.split(". ")
        if len(_strong) == 2:
            terms[_strong[0]] = _strong[1]
        else:
            errors.append(strong)
        
    else:
        errors.append(strong)

In [ ]:
terms